In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.STD)

// Create basic open Petri net
val model = Network("SPNBasicOpen")

// Create nodes representing Petri net elements
val source = Source(model, "TokenSource") // Token generation
val place1 = Queue(model, "Place1", SchedStrategy.INF)
val place2 = Queue(model, "Place2", SchedStrategy.FCFS) 
val sink = Sink(model, "TokenSink") // Token consumption

// Create open class for tokens
val tokenClass = OpenClass(model, "Tokens")

In [ ]:
// Set arrival rate (token generation rate)
source.setArrival(tokenClass, Exp.fitMean(1.5)) // Tokens arrive at rate 1.5

// Set service times (transition firing rates)
place1.setService(tokenClass, Exp.fitMean(2.0)) // Fast transition
place2.setService(tokenClass, Exp.fitMean(1.0)) // Slower transition

In [ ]:
// Set routing (Petri net structure)
model.addLink(source, place1)  // Token arrival to Place1
model.addLink(place1, place2)  // Transition Place1 -> Place2
model.addLink(place2, sink)    // Token departure from Place2

In [ ]:
// Solve with different solvers
val solvers = mutableListOf<Pair<String, NetworkSolver>>()

// CTMC solver with cutoff for open networks
try {
    val solverCtmc = CTMC(model)
    solverCtmc.setOptions("cutoff", 15) // Limit state space
    solvers.add("CTMC" to solverCtmc)
} catch (e: Exception) {
    println("CTMC solver error: ${e.message}")
}

// JMT solver (good for open networks)
try {
    val solverJmt = JMT(model)
    solverJmt.setOptions("samples", 100000)
    solvers.add("JMT" to solverJmt)
} catch (e: Exception) {
    println("JMT solver not available: ${e.message}")
}

In [ ]:
// Print results for all solvers
for ((name, solver) in solvers) {
    try {
        println("\n=== $name Solver Results ===")
        val avgTable = solver.avgTable
        avgTable.print()
        
        // Open Petri net analysis
        val place1Tokens = avgTable.getQLen().get(0, 1) // Average tokens in Place1
        val place2Tokens = avgTable.getQLen().get(0, 2) // Average tokens in Place2
        val arrivalRate = avgTable.getArvR().get(0, 0)  // Token arrival rate
        val throughput = avgTable.getTput().get(0, 3)   // Token departure rate
        
        println("\nOpen Petri Net Analysis:")
        println("Average tokens in Place1: $place1Tokens")
        println("Average tokens in Place2: $place2Tokens")
        println("Total tokens in system: ${place1Tokens + place2Tokens}")
        println("Token arrival rate: $arrivalRate")
        println("Token departure rate: $throughput")
        println("System utilization: ${throughput / arrivalRate}")
        
    } catch (e: Exception) {
        println("$name solver failed: ${e.message}")
    }
}